In [192]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

分析10天上涨超过8的机会, 在何时出现
- sz_jt_10： 0,1,2,3： 0, 0-3，3-8, 8-...  
sql 
- "SELECT * from day_zhibiao where 
- jx_days_ud60 > 20 and jx_days_ud60 < 40 and \
- jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 and \
- lj_fl_1_3 > 1.1 and lj_fl_3_10>1.05 "

In [193]:
sql = "SELECT * \
from ds_qf60"
print(sql)
df = pd.read_sql_query(sql, conn, index_col=None)

df.shape

SELECT * from ds_qf60


(35170, 47)

In [194]:
# df.info()

In [195]:
col_list_ds = ['symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'sz_jt_10','class0' ]
df.tail(1)[col_list_ds]

,symbol,trade_date,pct_change,diff,dea,bar,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,jx_zs_60,pct_fl_1_3,pct_fl_3_10,pct_fl_5_20,pct_fl_20_60,sz_max_5,sz_max_10,sz_max_20,sz_jt_10,class0
35169,689009,2021-04-19,3.04,-0.461137,-1.14476,1.36724,39,0.0,-0.143429,0.522034,...,0,1.30432,1.13952,0.745176,0.524615,0.379001,4.43571,4.43571,2.0,2


In [196]:
df['sz_jt_10'].value_counts(), df['sz_jt_10'].value_counts()/len(df)

(2.0    10830
 1.0    10013
 3.0     7440
 0.0     6887
 Name: sz_jt_10, dtype: int64,
 2.0    0.307933
 1.0    0.284703
 3.0    0.211544
 0.0    0.195820
 Name: sz_jt_10, dtype: float64)

In [197]:
# 开发测试：条件取数
# sum = len(df)

# cond2a = df['sz_jt_10']>=2
# df[ cond2a ]['sz_jt_10'].value_counts()

# cond2b = df['sz_jt_10']>1 
# cond2c = df['sz_jt_10']<=2
# cond2bc2 = ( cond2b & cond2c )
# df[ cond2bc2 ]['sz_jt_10'].value_counts()

In [198]:
steps = 5
steps_dg_5, steps_dg_10 = 11,1
steps_fl_1_3, steps_fl_3_10  = 1, 10

In [199]:
# jx_dg_5 预处理
min_dg_5, max_dg_5 = df['jx_dg_5'].min(), df['jx_dg_5'].max()*0.5
step_dg_5 = (max_dg_5 - min_dg_5)/steps_dg_5
# from_dg_5 = min_dg_5
# min_dg_5, max_dg_5, step_dg_5, from_dg_5  

In [200]:
# fl_1_3 处理
min_fl_1_3, max_fl_1_3 = df['pct_fl_1_3'].min(), df['pct_fl_1_3'].max()*0.2
step_fl_1_3 = (max_fl_1_3 - min_fl_1_3)/steps_fl_1_3
from_fl_1_3 = min_fl_1_3
min_fl_1_3, max_fl_1_3, from_fl_1_3, step_fl_1_3 


(1.15793, 13.264740000000002, 1.15793, 12.106810000000001)

In [201]:
# epoch 初始化数据
# col = ['pct_change',
#     'diff', 'dea',\
#     'jx_days_ud60',\
#     'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
#     'jx_zs_5', 'jx_zs_10', 'jx_zs_20', \
#     'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
#     'sz_max_10' ]
col = ['jx_dg_5','pct_fl_1_3','step', 'class', 'rate' ]
def header():
    print('- jx_days_ud60 > 20 and jx_days_ud60 < 40 ')
    print('- jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 ')
    print('- lj_fl_1_3 > 1.1 and lj_fl_3_10>1.05 ')
    print('-'*50)
    print('Loop: %8s,%8s, %8s,%8s'%('5日均线','5日均线','1-3日放量','1-3日放量'),'= %6s, %6s, %6s'%('数量','达成数','成功率%'))


In [202]:
%%time
# man[col] = 0.0
man_data = {'jx_dg_5':0.0,
            'pct_fl_1_3':0.0,
            'step':0, 'class':0, 'rate':0.0    
}
man = pd.DataFrame(data=man_data, index=[0])
header()
all_step, all_class = 0, 0
# 开始epoch
cond_begin = df['sz_jt_10']>=3

from_dg_5 = min_dg_5
for i_dg_5 in range(steps_dg_5):
    to_dg_5 = min_dg_5 + (i_dg_5+1)*step_dg_5
    # 处理条件
    cond_dg_5 = (df['jx_dg_5'] >= from_dg_5) & (df['jx_dg_5'] < to_dg_5 )
    # cond_fl_1_3_class = ( cond_fl_1_3 ) & (cond_last)
    cond_last = (cond_dg_5) & ( cond_begin )
    
    from_fl_1_3 = min_fl_1_3
    for i_fl_1_3 in range(steps_fl_1_3):
        to_fl_1_3 = min_fl_1_3 + (i_fl_1_3+1)*step_fl_1_3
        # print('%.0f : , %.4f,  %.4f'%(i, from_fl_1_3, to_fl_1_3), end=' ' )
        # 处理条件
        cond_fl_1_3 = (df['pct_fl_1_3'] >= from_fl_1_3) & (df['pct_fl_1_3'] < to_fl_1_3 )
        cond_fl_1_3_class = ( cond_fl_1_3 ) & (cond_last)
        
        # 开始计算   
        sum_step_series  = df[ cond_fl_1_3 ]['sz_jt_10'].value_counts() 
        sum_class_series = df[ cond_fl_1_3_class ]['sz_jt_10'].value_counts() 
        # 
        sum_step = sum_step_series.iloc[0] if not sum_step_series.empty else 0
        sum_class = sum_class_series.iloc[0] if not sum_class_series.empty else 0
        sum_class_rate = 0 if sum_class==0 else sum_class*100/sum_step
        print( i_dg_5, i_fl_1_3, ': %10.4f, %10.4f, %10.4f, %10.4f'%(from_dg_5, to_dg_5, from_fl_1_3, to_fl_1_3),\
            '= %8.0f, %8.0f, %8.2f'%(sum_step, sum_class, sum_class_rate) )
        # 更新到man
        all_step  = all_step + sum_step
        # all_step  += sum_step
        all_class += sum_class
        man_list = [from_dg_5, from_fl_1_3, sum_step, sum_class, sum_class_rate]
        man = pd.DataFrame(np.insert(man.values, len(man.index), values=man_list, axis=0))        
        # new_data = {'rate': sum_class_rate,'jx_dg_5': from_dg_5, 'pct_fl_1_3': from_fl_1_3}
        # man = man.append(new_data, ignore_index=True)

        # 下一个loop
        from_fl_1_3 += step_fl_1_3
    # jx_dg_5下一个loop
    from_dg_5 += step_dg_5
    
all_class_rate = all_class*100/all_step 
print( '-'*90)
print( ' '*52, '= %8.0f, %8.0f, %8.2f'%(all_step, all_class, all_class_rate) )
man.shape


- jx_days_ud60 > 20 and jx_days_ud60 < 40 
- jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 
- lj_fl_1_3 > 1.1 and lj_fl_3_10>1.05 
--------------------------------------------------
Loop:     5日均线,    5日均线,   1-3日放量,  1-3日放量 =     数量,    达成数,   成功率%
0 0 :    -1.8567,    -1.4056,     1.1579,    13.2647 =    10828,        0,     0.00
1 0 :    -1.4056,    -0.9545,     1.1579,    13.2647 =    10828,        6,     0.06
2 0 :    -0.9545,    -0.5034,     1.1579,    13.2647 =    10828,       66,     0.61
3 0 :    -0.5034,    -0.0523,     1.1579,    13.2647 =    10828,      465,     4.29
4 0 :    -0.0523,     0.3988,     1.1579,    13.2647 =    10828,     2703,    24.96
5 0 :     0.3988,     0.8499,     1.1579,    13.2647 =    10828,     2540,    23.46
6 0 :     0.8499,     1.3011,     1.1579,    13.2647 =    10828,     1077,     9.95
7 0 :     1.3011,     1.7522,     1.1579,    13.2647 =    10828,      391,     3.61
8 0 :     1.7522,     2.2033,     1.1579,    13.2647 =    10828,      121,     1.

(12, 5)

In [203]:
pd.set_option('display.max_rows', None)
man.columns = col
# man.sort_index(ascending=True)
# man.set_index('rate', drop=False)
sorting = man.sort_values('rate', ascending=False)
sorting

,jx_dg_5,pct_fl_1_3,step,class,rate
5,-0.052277,1.15793,10828.0,2703.0,24.963059
6,0.398836,1.15793,10828.0,2540.0,23.457702
7,0.849949,1.15793,10828.0,1077.0,9.946435
4,-0.503390,1.15793,10828.0,465.0,4.294422
8,1.301062,1.15793,10828.0,391.0,3.611008
9,1.752175,1.15793,10828.0,121.0,1.117473
3,-0.954504,1.15793,10828.0,66.0,0.609531
10,2.203289,1.15793,10828.0,42.0,0.387883
11,2.654402,1.15793,10828.0,17.0,0.157000
2,-1.405617,1.15793,10828.0,6.0,0.055412


In [204]:
'-'*10

'----------'

In [205]:
# man